In [1]:
import cv2
import os,sys
import numpy as np
import csv
import pandas as pd
from PIL import Image

# ATTENDANCE MANAGEMENT THROUGH FACE RECOGNITION

# ==========REGISTERATION==========

In [2]:

import cv2
import os,sys
import numpy as np
import csv
name = input("Enter name")
choice  = input("1:Images from camera\n2:images from gallery\n")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Id = 0

if os.path.exists("AttendanceSheet.csv"):
    num_rows= 0
    for row in open("AttendanceSheet.csv"):
        print(num_rows)
        num_rows += 1
    Id = num_rows
else:
    Id = "1"

    
if choice == "1":
    # body to get images from camera
    camera = cv2.VideoCapture(0)
    sample_num = 0
    while True:
        ret,frame = camera.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                cv2.putText(frame,str(sample_num),(10,90),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                sample_num += 1
                cv2.imwrite("image_capture/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])
        cv2.imshow("frame",frame)
        if cv2.waitKey(100) & 0xFF == 27:
            break
    camera.release()
    cv2.destroyAllWindows()    
    
elif choice == "2":    
    path = "burak/"
    images_paths = [os.path.join(path,f) for f in os.listdir(path)]
    sample_num = 0
    for image in images_paths:
        image = cv2.imread(image)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():
            for (x,y,w,h) in faces:
                sample_num += 1
                cv2.imwrite("image_capture/"+name +"."+str(Id) +'.'+str(sample_num) + ".jpg",gray[y:y+h,x:x+w])
row = [name,Id]  
if os.path.exists("AttendanceSheet.csv"):
    with open("AttendanceSheet.csv",'a+',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)
        csvfile.close()
else:
    with open("AttendanceSheet.csv",'w',newline = '') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['name','Id'])
        writer.writerow(row)
        csvfile.close()           

Enter nameosama
1:Images from camera
2:images from gallery
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


# ==============TRAINER==================

In [3]:
import sys,os
import numpy as np
from PIL import Image

faces=[]
Ids=[]
imagePaths=[os.path.join("image_capture/",img) for img in os.listdir("image_capture/")]

for path in imagePaths:
        pilImage = Image.open(path).convert('L')
        imageNp=np.array(pilImage,'uint8')
        Id = int(os.path.split(path)[-1].split(".")[1])
        faces.append(imageNp)
        Ids.append(Id)

if os.path.exists("model_hafsa.yml"):
    recognizer = cv2.face_LBPHFaceRecognizer.create()
    recognizer.read("model_hafsa.yml")
else:
    recognizer = cv2.face_LBPHFaceRecognizer.create()
        
recognizer.train(faces,np.array(Ids))
recognizer.save("model_hafsa.yml")
print("Model for attendance is trained! ")

Model for attendance is trained! 


# ================PREDICTOR==============

In [3]:

import cv2

image = cv2.imread("sajal.jpg")
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read("model_hafsa.yml")
Id,conf = recognizer.predict(image)
print(Id)

12


# ==============RECOGNIZER=============

In [ ]:
import cv2
import pandas as pd
from datetime import datetime 
# import os 

recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read("model_hafsa.yml")
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
df = pd.read_csv("AttendanceSheet.csv")

camera = cv2.VideoCapture(0)

#=======================In this body *attendance* is marked of students who are present==================================== 

def markAttendance(name):
    with open('AttendancePresent.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        #print(myDataList)
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')  
    
    
namelist=[]                        
#The code is doing recognition of faces with live cam 
while True:
    ret,frame = camera.read()
    if ret:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray,1.2,5)
        if faces is not ():    
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
                Id,conf = recognizer.predict(gray[y:y+h,x:x+w])
                if (conf > 10):
                    name = df.loc[df["Id"] == Id]['name'].values
                    
                    if(len(name) != 0):
                        namelist.append(str(name))
                        print("Name : ",str(name))
                        markAttendance(str(name))
                        
                    cv2.putText(frame,str(name),(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                else:
                    cv2.putText(frame,"face unknown",(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
                cv2.imshow("frame",frame)
        else:
            print("no face found")
    else:
        print("no frame fetch")
    if cv2.waitKey(100) & 0xFF == 27:
        break
camera.release()
cv2.destroyAllWindows()

Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
Name :  ['osama']
no face found
no face found
no face found
no face found
no face found
no face found
Name :  ['osama shafiq']
no face found
Name :  ['mahira']
Name :  ['fahad']
Name :  ['fahad']
Name :  ['fahad']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :  ['sajal']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :  ['fahad']
Name :  ['hafsa junaid']
Name :

no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
Name :  ['sajal']
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no face found
no

In [20]:
namelist = list(dict.fromkeys(namelist))
print(namelist)

["['mahira']", "['hafsa junaid']", "['sajal']", "['faizi']", "['osama shafiq']"]


# MANAGEMENT OF ATTENDANCE 

In [30]:
import os
import pandas as pd
from datetime import date,datetime,timedelta
import xlsxwriter
from openpyxl import *
import sys

def allDatesOfCurrentMonth():
    m = datetime.now().month
    y = datetime.now().year
    ndays = (date(y, m+1, 1) - date(y, m, 1)).days
    d1 = date(y, m, 1)
    d2 = date(y, m, ndays)
    delta = d2 - d1

    return [(d1 + timedelta(days=i)).strftime('%d-%m-%Y') for i in range(delta.days + 1)]

listOfAllDatesOfCurrentMonth = allDatesOfCurrentMonth()            #Global date list variable

def starting():
    print("\t\tWelcome to our Face Recognition attendence System\n\n")
    subject = input("Enter the name of the Subject : (P for python & J for Java) -1 to exit\n")

    if subject == '-1':
        return 0

    if subject == 'P' or subject == 'J':
        return subject

    else:
        return 1

def getAndCheckDate():
    currentMonth = datetime.now().month
    currentYear = datetime.now().year
    time = str(currentMonth) + "-" + str(currentYear)
    date = input("Enter any date of the month-year(%s" % time + ")\n")
    time = str(date) + "-" + str(currentMonth) + "-" + str(currentYear)

    userEnterDate = int(time.split('-')[0])
    monthStartingDate = int(listOfAllDatesOfCurrentMonth[0].split('-')[0])
    monthEndingDate = int(listOfAllDatesOfCurrentMonth[-1].split('-')[0])
    if userEnterDate >= monthStartingDate and userEnterDate <= monthEndingDate:
        return time
    else:
        return 0

def getAttendenceListOfAllTheStudents():                        #Return list of the attendence student
                                           #Now make a attendence List...

    print(namelist)
    df = pd.read_csv("C:/Users/Hafsa Junaid/Documents/Attendance through face-recognition/AttendanceSheet.csv")
    # Make a list with student ID , Name , Status(Present , Absent)
    attendenceList = []
    for index, row in df.iterrows():
        user_email = str(row['name'])
        first="['"
        last="']"
        user_email=first+user_email+last
        result=user_email in namelist
        print(result)
        if result==True:
             attendenceList.append([row['Id'], row['name'], "P"])
        else:
             attendenceList.append([row['Id'], row['name'], "A"])
            
            

       
                

        
    return attendenceList           #Having Student ID Student Name and Student status...

def checkExcelFileAlreadyExistOrNot():
    filesList = os.listdir('C:/Users/Hafsa Junaid/Documents/Attendance through face-recognition/')
    result = any(filesList in "Attendence.xlsx" for filesList in filesList)

    return result

def createNewExcelFileAndFillIt(attendenceList , subject , time):
    # Create a new Excel File...
    workbook = xlsxwriter.Workbook('Attendence.xlsx')
    # For python...
    worksheet1 = workbook.add_worksheet('Python')
    # For Java
    worksheet2 = workbook.add_worksheet('Java')

    # Insert all the names in both worksheet...

    row = 1
    for names in attendenceList:
        worksheet1.write(row, 0, names[1])
        worksheet2.write(row, 0, names[1])
        row = row + 1

    # Now insert all the dates in both work sheets of current month...
    
    col = 1
    for dates in listOfAllDatesOfCurrentMonth:
        worksheet1.write(0, col, dates)
        worksheet2.write(0, col, dates)
        col = col + 1


    col = int(time.split('-')[0])
    row = 1
    if subject == 'P':
        for names in attendenceList:
            worksheet1.write(row, col, names[2])
            row = row + 1
    elif subject == 'J':
        for names in attendenceList:
            worksheet2.write(row, col, names[2])
            row = row + 1

    workbook.close()

def readExistingExcelFileAndFillIt(attendenceList , subject , time):
    wb = load_workbook("C:/Users/Hafsa Junaid/Documents/Attendance through face-recognition/Attendence.xlsx")
    workSheet1 = wb['Python']
    workSheet2 = wb['Java']

    col = int(time.split('-')[0]) + 1
    row = 2
    if subject == 'P':
        for names in attendenceList:
            wcell1=workSheet1.cell(row,col)
            wcell1.value = names[2]
            row = row + 1
    elif subject == 'J':
        for names in attendenceList:
            wcell1 = workSheet2.cell(row, col)
            wcell1.value = names[2]
            row = row + 1

    wb.save("C:/Users/Hafsa Junaid/Documents/Attendance through face-recognition/Attendence.xlsx")

    print("Read Existing Excel File\n")

def run():
    subject = starting()
    
    if(subject == 0):

        print("Thanks for using our Program\n")
        sys.exit()

    elif(subject == 1):

        print("Run the Program again.\nThank you.\n")

    else:
                                                #Get the Subject Name here...
        date = getAndCheckDate()

        if date == 0:

            print("User enter invalid Date.\nPlease run the program Again.\n")

            sys.exit()

        else:
                                                # Get the Valid Date here...
            attendenceList = getAttendenceListOfAllTheStudents()
                                                #Get Attendence List here...

            fileExist = checkExcelFileAlreadyExistOrNot()

            if fileExist == True:
                readExistingExcelFileAndFillIt(attendenceList, subject, date)


            else:
                createNewExcelFileAndFillIt(attendenceList , subject , date)
                print("Create a new file and fill it\n")
run()

		Welcome to our Face Recognition attendence System


Enter the name of the Subject : (P for python & J for Java) -1 to exit
P
Enter any date of the month-year(1-2021)
7
["['mahira']", "['hafsa junaid']", "['sajal']", "['faizi']", "['osama shafiq']"]
True
True
False
False
False
False
False
False
False
True
True
True
False
False
False
False
False
False
Read Existing Excel File

